In [6]:
import mlrun.feature_store as fs
import mlrun

In [7]:
mlrun.set_environment(project="stocksdani")

('stocksdani', 'v3io:///projects/{{run.project}}/artifacts')

In [8]:
features = ["stocks.Closes_min_1h",
            "stocks.Closes_max_1h",
            "stocks.Volumes_min_1h",
            "stocks.Volumes_max_1h",
            "stocks.Open",
            "stocks.High",
            "stocks.Low",
            "stocks.Close",
            "stocks.Volume",
#                 "news.Content",
#                 "news.Link",
            "news.Sentiment"]

In [9]:
vector = fs.FeatureVector("stocks-vec", features, description="stocks demo feature vector")
vector.save()

In [12]:
df = fs.get_offline_features("stocks-vec").to_dataframe().dropna()

In [18]:
import pandas as pd
from mlrun import MLClientCtx
import os
import mlrun.feature_store as fs
import mlrun
import numpy as np
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from datetime import datetime
import keras
import nuclio

In [33]:
df

,Closes_min_1h,Closes_max_1h,Volumes_min_1h,Volumes_max_1h,Open,High,Low,Close,Volume,Sentiment
13507,262.690002,262.690002,646468.0,646468.0,262.720001,263.480011,262.399994,262.690002,646468,0.0
13515,262.609985,262.690002,77058.0,646468.0,262.739990,263.058990,262.600006,262.609985,77058,0.0
13520,262.609985,262.999908,77058.0,646468.0,262.600006,263.170013,262.589996,262.999908,85948,0.0
13524,262.609985,263.269989,50055.0,646468.0,263.015015,263.279999,262.989990,263.269989,50055,0.0
13528,262.609985,263.269989,50055.0,646468.0,263.250000,263.429993,263.149994,263.239990,52618,0.0
...,...,...,...,...,...,...,...,...,...,...
15419,265.079987,265.769989,25013.0,231546.0,265.380005,265.420013,265.315002,265.359985,113474,0.0
15424,2439.750000,2448.000000,721.0,17858.0,2447.459961,2447.709961,2447.179932,2447.179932,17549,0.0
15425,265.079987,265.769989,25013.0,231546.0,265.350006,265.364990,265.130005,265.130005,118529,0.0
15428,265.079987,265.769989,25013.0,283208.0,265.130005,265.559998,265.109985,265.510010,283208,0.0


In [29]:
y_train = df["Close"]
X_train = df.drop(["Close"],axis = 1).to_numpy()
X_train = np.reshape(X_train, (X_train.shape[0], 1, df.drop(["Close"],axis = 1).shape[1]))

In [30]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units =  1))

#Compiling and fitting the model
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# regressor.fit(X_train, y_train, epochs = 2, batch_size = 32)

In [31]:
regressor.fit(X_train, y_train, epochs = 2, batch_size = 32)

Epoch 1/2
14/14 [==============================] - 0s 22ms/step - loss: 668990.3750
Epoch 2/2
14/14 [==============================] - 0s 29ms/step - loss: 668871.6875
